In [3]:
import spacy
from spacy.lang.ru.examples import sentences
from spacy import displacy
from pprint import pprint
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np

In [4]:
nlp = spacy.load("ru_core_news_sm")

In [5]:
newsDF = pd.read_csv('hrefs.CSV')
text_news = newsDF['text']
text_news = np.array(text_news)
text_news[2]

'\nВ столичном регионе в конце мая – начале июня чаще всего фиксируются шквалистые ветры и ураганы, однако на данный момент погода спокойная. Об этом в среду, 24 мая, рассказал замдиректора Института физики атмосферы имени Обухова Российской академии наук Владимир Семенов.\n"Вторая половина мая и первая половина июня – это время, когда наиболее сильный температурный контраст между северными морями и европейской частью России. Поэтому если случаются холодные вторжения, то именно в это время чаще всего бывают шквалы, сильный порывистый ветер, как в 2017 году было в Москве", – пояснил специалист в беседе с агентством городских новостей "Москва".\nПри этом пока что погода в столице дождливая, а сильной жары не было. В связи с этим в ближайшие несколько дней ураганов не будет. Обычно их следует ждать после периода жаркой погоды, когда воздух наполняется влагой, заключил Семенов.\nСогласно прогнозам метеорологов, в ближайшие выходные в Москву придет похолодание. А метеорологическое лето в ст

In [6]:
doc = nlp(text_news[11])
for s in list(doc.sents)[:2]:
  pprint(list([f'{t.lemma_} ({t.text})' for t in s]))

['\n (\n)',
 'в (В)',
 'столичный (столичном)',
 'регион (регионе)',
 'в (в)',
 'конец (конце)',
 'май (мая)',
 '– (–)',
 'начало (начале)',
 'июнь (июня)',
 'чаще (чаще)',
 'всего (всего)',
 'фиксироваться (фиксируются)',
 'шквалистый (шквалистые)',
 'ветер (ветры)',
 'и (и)',
 'ураган (ураганы)',
 ', (,)',
 'однако (однако)',
 'на (на)',
 'данный (данный)',
 'момент (момент)',
 'погода (погода)',
 'спокойный (спокойная)',
 '. (.)']
['Об (Об)',
 'это (этом)',
 'в (в)',
 'среда (среду)',
 ', (,)',
 '24 (24)',
 'май (мая)',
 ', (,)',
 'рассказать (рассказал)',
 'замдиректора (замдиректора)',
 'институт (Института)',
 'физика (физики)',
 'атмосфера (атмосферы)',
 'имя (имени)',
 'обухов (Обухова)',
 'российский (Российской)',
 'академия (академии)',
 'наука (наук)',
 'владимир (Владимир)',
 'семёнов (Семенов)',
 '. (.)',
 '\n (\n)']


In [7]:
for s in list(doc.sents)[:2]:
  displacy.render(s, style="dep", minify=True, jupyter=True, options={"distance":90})

In [18]:
sent0 = list(doc.sents)[0]
childs0 = list(sent0.root.children)
print(sent0.root, childs0)
# Загнать этот вывод в цикл и проанализировать все текста?
print(childs0[0].lemma_, childs0[0].text, childs0[0].dep_, childs0[0].tag_, '->', list(childs0[0].ancestors)[0])

фиксируются [регионе, конце, чаще, ветры, спокойная, .]
регион регионе obl NOUN -> фиксируются


In [22]:
with open("rusentilex_2017.txt", encoding="utf-8") as senti_file: 
    for i in range(18): # Пропускаем заголовок файла.
        _ = senti_file.readline()
        
    senti_words = {line.split(", ")[0]: line.split(", ")[3] for line in senti_file.readlines()}

In [23]:
def find_sentiments_spacy(node, senti_words, sentiments):
    for child in node.children:
        if child.lemma_ in senti_words.keys():
            if node.lemma_ not in sentiments.keys():
                sentiments[node.lemma_] = defaultdict(int)
            sentiments[node.lemma_][senti_words[child.lemma_]] += 1

In [24]:
sentiments = {}
for sent in doc.sents:
    find_sentiments_spacy(sent.root, senti_words, sentiments)
sentiments

{'фиксироваться': defaultdict(int, {'positive': 1}),
 'бывать': defaultdict(int, {'neutral': 1}),
 'быть': defaultdict(int, {'negative': 1}),
 'будет': defaultdict(int, {'neutral': 1}),
 'прийти': defaultdict(int, {'positive/negative': 1}),
 'стартовать': defaultdict(int, {'negative': 1})}

In [25]:
sentiments = {}
for sent_text in tqdm(text_news):
    doc = nlp(sent_text)
    for sentence in doc.sents:
        find_sentiments_spacy(sentence.root, senti_words, sentiments)

100%|██████████| 408/408 [00:21<00:00, 18.94it/s]


In [30]:
[(senti, vals) for senti,vals in sentiments.items() if vals['negative']>10], '\n----\n',\
[(senti, vals) for senti,vals in sentiments.items() if vals['positive']>10]

([('получить', defaultdict(int, {'negative': 19, 'positive': 4})),
  ('быть', defaultdict(int, {'negative': 14, 'positive': 4, 'neutral': 3})),
  ('заявить',
   defaultdict(int,
               {'negative': 12,
                'positive': 29,
                'neutral': 5,
                'positive/negative': 1})),
  ('мочь',
   defaultdict(int,
               {'negative': 14,
                'positive': 11,
                'positive/negative': 5,
                'neutral': 1,
                'пострадать': 1})),
  ('действовать',
   defaultdict(int, {'negative': 12, 'neutral': 1, 'positive': 0}))],
 '\n----\n',
 [('выразить', defaultdict(int, {'positive': 12, 'negative': 5})),
  ('заявить',
   defaultdict(int,
               {'negative': 12,
                'positive': 29,
                'neutral': 5,
                'positive/negative': 1})),
  ('отметить', defaultdict(int, {'negative': 1, 'positive': 11})),
  ('сообщить',
   defaultdict(int,
               {'negative': 7,
            